In [7]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

d:\coding\AI-Based_Psychological_Counseling_Assistant\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_csv("Mental_Health_FAQ.csv")

In [4]:
data.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [8]:
# Initialize the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

d:\coding\AI-Based_Psychological_Counseling_Assistant\venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rushi\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
# Generate embeddings for questions
print("Generating embeddings...")
embeddings = embedding_model.encode(data['Questions'].tolist())

# Convert embeddings to a NumPy array
embeddings = np.array(embeddings, dtype='float32')


Generating embeddings...


In [10]:
# Initialize FAISS index
print("Initializing FAISS index...")
dimension = embeddings.shape[1]  # Length of embeddings
index = faiss.IndexFlatL2(dimension)  # L2 distance metric

Initializing FAISS index...


In [11]:
# Add embeddings to the index
print("Adding embeddings to FAISS index...")
index.add(embeddings)

Adding embeddings to FAISS index...


In [28]:
# Save the FAISS index to a file
faiss.write_index(index, "faiss_index_file.index")
print("FAISS index saved successfully!")

FAISS index saved successfully!


In [29]:
# Load the FAISS index
index = faiss.read_index("faiss_index_file.index")
print("FAISS index loaded successfully!")

# Query
query = "I feel anxious in public speaking? What should I do?"
query_embedding = np.array(embedding_model.encode([query]), dtype='float32')

# Search for similar contexts
print("Searching for similar contexts...")
k = 4  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_embedding, k)

# Retrieve top contexts
results = []
for i in indices[0]:
    if i < len(data):
        results.append({
            'question': data.iloc[i]['Questions'],
            'answer': data.iloc[i]['Answers']
        })

# Display retrieved contexts
print("\nTop Matching Contexts:")
for res in results:
    print(f"Q: {res['question']}\nA: {res['answer']}\n")

# Combine results for context text
context_text = "\n".join([f"Q: {res['question']}\nA: {res['answer']}" for res in results])
print("\nCombined Context for Query:")
print(context_text)

FAISS index loaded successfully!
Searching for similar contexts...

Top Matching Contexts:
Q: How can I use distraction to manage difficult thoughts or feelings?
A: Distraction is a very valid tool to help you cope when everything feels overwhelming or when you feel lonely or isolated. 
 If you don't have a lot of energy or focus right now, try low-effort distractions like watching TV, browsing Youtube, listening to a podcast or audiobook, playing a game on your phone, reading an easy book or magazine, or working on a simple art project. 
 If you have more energy and focus, give yourself a to-do list every day: you can clean and take care of projects around your home, work on hobbies, connect with family or friends, read a new book and catch up on your favourite TV shows. You can find interesting opportunities to take online courses from universities all over the world through MOOCs and other online learning platforms, you can learn a new language online or through apps, and you can le

In [24]:
from dotenv import load_dotenv
import os

# Load variables from the .env file
load_dotenv()

api_key = os.getenv('API_KEY')
print(api_key)

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


In [25]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=1,
    groq_api_key = api_key
)

In [26]:
# Construct the prompt combining context and the user's query
prompt = f"""
You are a mental health assistant providing thoughtful and empathetic responses based on a context of similar questions and answers.

Context:
{context_text}

User's Question:
{query}

Provide a helpful and supportive response based on the context.
"""


In [27]:

# Generate the response
response = llm.invoke(prompt)

# Print the response
print(response)

content="It's completely normal to feel anxious when faced with public speaking. Many people experience anxiety in high-pressure situations like this, and it's not necessarily something to be ashamed of. \n\nOne helpful approach when dealing with public speaking anxiety is to focus on distractions. Instead of ruminating on your anxiety or worrying about what others might think, try to focus on what you're doing or saying. This can help take your mind off your anxiety and distract you from it. For example, you can think about the content of your presentation, practice what you're going to say, or focus on the audience you're speaking to.\n\nTo get even more comfortable, consider building your confidence by taking small steps. Begin by practicing public speaking in low-stakes situations, like speaking up in a small meeting or presenting an idea to a friend or family member. As you become more confident, you can gradually move on to more significant public speaking situations.\n\nIt's als